In [2]:
import os
import cv2 
import time

In [3]:
#TRABAJANDO EN DIRECTORIO 
#os.chdir(r'C:\VIDEO')

PASOS GENERALES:
1- Conversión a escala de grises y eliminación de ruido.
2- Operación de substracción entre el segundo plano y el primer plano.
3- Aplicar un umbral a la imagen resultado de la resta.
4- Detección de contornos.

In [4]:
# Cargamos el vídeo
#VideoCapture.read(). La función devuelve el cuadro de video. Si no se han tomado fotogramas, la imagen estará vacía.
camara = cv2.VideoCapture("GATOS.mp4")


In [5]:
#Vemos el video
while (camara.isOpened()):
    ret, imagen = camara.read()
    if ret == True:
        cv2.imshow('video', imagen)
        if cv2.waitKey(30) == ord('s'):
            break
    else:
        break

In [6]:
camara.release()
cv2.destroyAllWindows()

In [11]:
camara = cv2.VideoCapture("GATOS.mp4")
# Inicializamos el primer frame a vacío.
# Nos servirá para obtener el fondo
fondo = None

# Recorremos todos los frames
while True:
    (ret, frame) = camara.read()
    
    #Si viene rotada, se repara en los grados necesarios
    frame = cv2.rotate(frame, cv2.ROTATE_180)
    #frame = cv2.rotate(frame, cv2.ROTATE_90_CLOCKWISE)
    #frame = cv2.rotate(frame, cv2.ROTATE_90_COUNTERCLOCKWISE)

	# Si hemos llegado al final del vídeo salimos
    if not ret:
        break
    
	# Convertimos a escala de grises
    gris = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

	# Aplicamos suavizado para eliminar ruido
    #https://docs.opencv.org/3.4/d4/d13/tutorial_py_filtering.html
    gris = cv2.GaussianBlur(gris, (21, 21), 0)

	# Si todavía no hemos obtenido el fondo, lo obtenemos
	# Será el primer frame que obtengamos
    if fondo is None:
        fondo = gris
        continue

	# Calculo de la diferencia entre el fondo y el frame actual
    #absdiff calcula la diferencia absoluta entre los elementos de una imagen o un escalar
    resta = cv2.absdiff(fondo, gris) #diferencia absoluta entre frames del video (matrices de imágenes)

	# Aplicamos un umbral
    #https://docs.opencv.org/3.4/d7/d4d/tutorial_py_thresholding.html
    #Si el valor de píxel es menor que el umbral, se establece en 0; de lo contrario, se establece en un valor máximo.
    umbral = cv2.threshold(resta, 25, 255, cv2.THRESH_BINARY)[1] #umbrales imagen SIEMPRE en escala de grises

    # Dilatamos el umbral para tapar agujeros
    #https://docs.opencv.org/3.4/d9/d61/tutorial_py_morphological_ops.html
    #Las transformaciones morfológicas son algunas operaciones simples basadas en la forma de la imagen.
    #Normalmente se realiza sobre imágenes binarias. Necesita dos entradas, una es nuestra imagen original, 
    #la segunda se llama elemento estructurante o núcleo que decide la naturaleza de la operación. 
    #Dos operadores morfológicos básicos son la erosión y la dilatación.
   
    #Dilatación: Aquí aumenta la región blanca en la imagen o aumenta el tamaño del objeto en primer plano. 
    umbral = cv2.dilate(umbral, None, iterations=2)

	# Copiamos el umbral para detectar los contornos
    contornosimg = umbral.copy()

	# Buscamos contorno en la imagen
    #https://docs.opencv.org/3.4/d4/d73/tutorial_py_contours_begin.html
    #CHAIN_APPROX_SIMPLE Elimina todos los puntos redundantes y comprime el contorno, ahorrando así memoria.
    contornos, hierarchy = cv2.findContours(contornosimg,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)

	# Recorremos todos los contornos encontrados
    for c in contornos:
        # Eliminamos los contornos más pequeños
        if cv2.contourArea(c) < 4000:
            continue

		# Obtenemos los límites del contorno, el rectángulo mayor que engloba al contorno
        #boundingRect es una función que se utiliza para crear un rectángulo aproximado junto con la imagen. 
        #El uso principal de esta función es resaltar el área de interés después de obtener la forma 
        #exterior de la imagen. Con las marcas adecuadas, los usuarios pueden resaltar fácilmente el 
        #aspecto deseado en una imagen. Por ejemplo, en el reconocimiento de rostros, 
        #después de reconocer el rostro, las imágenes se vuelven a dibujar utilizando un límite 
        #que cubre los rostros de las personas reconocidas en la foto
        (x, y, w, h) = cv2.boundingRect(c) #https://www.pythonpool.com/cv2-boundingrect/
		
        # Dibujamos el rectángulo del contorno
        #cv2.rectangle(image, start_point, end_point, color, thickness)
        #https://www.geeksforgeeks.org/python-opencv-cv2-rectangle-method/
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

	# Mostramos las imágenes de la cámara, el umbral y la resta
    cv2.imshow("Camara", frame)
    cv2.imshow("Umbral", umbral)
    cv2.imshow("Resta", resta)
    cv2.imshow("Contorno", contornosimg)

	# Capturamos una tecla para salir
    key = cv2.waitKey(1) & 0xFF

	# Tiempo de espera para que se vea bien
    time.sleep(0.05)

In [12]:
# Liberamos la cámara y cerramos todas las ventanas
camara.release()
cv2.destroyAllWindows()